# Waste Classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!unzip "waste classification.zip"

In [ ]:
import os
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/waste-classification/DATASET/TRAIN'
valid_path = '/content/drive/MyDrive/waste-classification/DATASET/TEST'

In [ ]:
# Import the Resnet50 architecture
efficientnetb7 = EfficientNetB7(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

258088960/258076736 [==============================] - 3s 0us/step


In [ ]:
for layer in efficientnetb7.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/waste-classification/DATASET/TRAIN/*')

In [ ]:
x = Flatten()(efficientnetb7.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

model_eff = Model(inputs=efficientnetb7.input, outputs=prediction)

In [ ]:
model_eff.summary()

In [ ]:
model_eff.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/waste-classification/DATASET/TRAIN',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 22564 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/waste-classification/DATASET/TEST',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2513 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
MODEL_DIR = "/content/drive/MyDrive/waste-classification"

if not os.path.exists(MODEL_DIR):  #If the directory does not exist, create it.
    os.makedirs(MODEL_DIR)
checkpoint = ModelCheckpoint(filepath=os.path.join(MODEL_DIR, "model-{epoch:02d}.h5"), save_best_only=True) 

In [212]:
#fit the model
r = model_eff.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/20
706/706 [==============================] - 628s 854ms/step - loss: 8.1392 - accuracy: 0.5065 - val_loss: 11.8317 - val_accuracy: 0.4425
Epoch 2/20
706/706 [==============================] - 591s 836ms/step - loss: 7.0871 - accuracy: 0.5074 - val_loss: 2.7616 - val_accuracy: 0.4425
Epoch 3/20
706/706 [==============================] - 589s 833ms/step - loss: 6.5524 - accuracy: 0.5090 - val_loss: 5.5231 - val_accuracy: 0.5575
Epoch 4/20
706/706 [==============================] - 589s 833ms/step - loss: 5.1367 - accuracy: 0.5129 - val_loss: 2.6848 - val_accuracy: 0.4425
Epoch 5/20
706/706 [==============================] - 589s 834ms/step - loss: 6.7185 - accuracy: 0.5058 - val_loss: 0.6948 - val_accuracy: 0.4588
Epoch 6/20
706/706 [==============================] - 589s 834ms/step - loss: 7.7479 - accuracy: 0.5112 - val_loss: 0.9900 - val_accuracy: 0.5575
Epoch 7/20
706/706 [==============================] - 589s 834ms/step - loss: 6.8121 - accuracy: 0.5088 - val_loss: 13.2536

KeyboardInterrupt: ignored

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss_eff')

# accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc_eff')

In [ ]:
from tensorflow.keras.models import load_model

model_eff.save('model_efficientnetv7.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:

y_pred = model_eff.predict(test_set)


In [ ]:
y_pred

array([[0.8395259 , 0.16047409],
       [0.9680526 , 0.03194739],
       [0.76154286, 0.23845714],
       ...,
       [0.4356893 , 0.56431067],
       [0.63472533, 0.36527467],
       [0.97221667, 0.02778335]], dtype=float32)

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model_eff=load_model('model_efficientnetv7.h5')

In [ ]:
img=image.load_img('/content/drive/MyDrive/waste-classification/DATASET/TEST/R/R_10003.jpg', target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [252., 255., 251.],
        [252., 255., 251.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [252., 255., 251.],
        [252., 255., 251.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model_eff.predict(img_data)

array([[3.1376642e-37, 1.0000000e+00]], dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a

array([1])